# Introduction to WaterTAP Costing

# DELETE THIS CELL BEFORE MERGING
Outline:
- Brief summary of WaterTAP costing - slides will hopefully cover most of the details
- Just build an RO flowsheet
- Walk through adding the existing costing for each unit model (consider showing snippets of the unit model costing files or documentation with our explanations)
- Walk through how to modify the existing parameters
- How to add system level metrics (LCOW, SEC, etc.)
- Walk through how to go from simulating costing to optimizing it (add objective function for minimizing LCOW)
- Add a try it yourself for optimizing for a different objective (SEC)
- Walk through how they might add their own costing (register their own flow types) as shown in #2 here: https://watertap.readthedocs.io/en/latest/technical_reference/costing/costing_base.html

## TODO: Mutka should look into adding capital cost components dynamically


This tutorial will demonstrate how to add costing to an RO unit model as well as how to compute and optimize system level metrics like levelized cost of water (LCOW), specific energy consumption (SEC), and annual water production.

## Step 1: Import Modules

In [ ]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Objective,
    value,
    assert_optimal_termination,
    TransformationFactory,
    units as pyunits,
)

# Imports from IDAES
from idaes.core import FlowsheetBlock

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.costing import WaterTAPCosting
from watertap.core.solvers import get_solver

## Step 2: Build RO Model

In [ ]:
# Build flowsheet essentials
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Build property model
m.fs.properties = SeawaterParameterBlock()

# Build unit model 
m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
)

# Specify inlet feed conditions (state variables)
m.fs.RO.inlet.flow_vol_phase["Liq"].fix(1e-3)
m.fs.RO.inlet.conc_mass_phase_comp["Liq", "TDS"].fix(35)
m.fs.RO.inlet.pressure.fix(101325)
m.fs.RO.inlet.temperature.fix(273.15 + 25)

# Specify RO operating conditions
m.fs.RO.A_comp.fix(4.2e-12)
m.fs.RO.B_comp.fix(3.5e-8)
m.fs.RO.recovery_vol_phase[0, "Liq"].fix(0.5)
m.fs.RO.feed_side.velocity[0, 0].fix(0.15)
m.fs.RO.feed_side.channel_height.fix(1e-3)
m.fs.RO.feed_side.spacer_porosity.fix(0.97)
m.fs.RO.permeate.pressure[0].fix(101325)

## Step 3: Add RO Costing

### Step 3a: Create the Costing Model

`WaterTAPCosting` contain extensions, methods, and variables and constraints common to all WaterTAP Costing Packages. In other words, this sets up the base cost model, which we will expand upon by adding RO-specific cost terms

## Step 4: Modify Existing Parameters

## Step 5: Add System-Level Metrics

## Step 6: Optimize LCOW

## Try It Yourself

## Step 7: Register Custom Flows